In [1]:

import json
import os
import pandas as pd
import pprint
import tensorflow as tf
import time
import numpy as np
from keras.backend import resize_images
from keras.layers import *
print(tf.__version__)

import cv2

C:\Users\Kiko-PC\Anaconda3\envs\tensorflow\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
Using TensorFlow backend.


1.12.0


In [3]:
from keras.models import load_model
from keras.applications.xception import preprocess_input


model = load_model('checkpoints/model-016.h5',  custom_objects={'preprocess_input': preprocess_input})

C:\Users\Kiko-PC\Anaconda3\envs\tensorflow\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
def load_image(path):
    image = cv2.cvtColor(cv2.imread(path,-1), cv2.COLOR_BGR2RGB)
    return image


In [5]:

def get_label_info(csv_path):
    """
    Retrieve the class names and label values for the selected dataset.
    Must be in CSV format!

    # Arguments
        csv_path: The file path of the class dictionairy
        
    # Returns
        Two lists: one for the class names and the other for the label values
    """
    filename, file_extension = os.path.splitext(csv_path)
    if not file_extension == ".csv":
        return ValueError("File is not a CSV!")

    class_names = []
    label_values = []
    with open(csv_path, 'r') as csvfile:
        file_reader = csv.reader(csvfile, delimiter=',')
        header = next(file_reader)
        for row in file_reader:
            class_names.append(row[0])
            label_values.append([int(row[1]), int(row[2]), int(row[3])])
        # print(class_dict)
    return class_names, label_values

def one_hot_it(label, label_values):
    """
    Convert a segmentation image label array to one-hot format
    by replacing each pixel value with a vector of length num_classes

    # Arguments
        label: The 2D array segmentation image label
        label_values
        
    # Returns
        A 2D array with the same width and hieght as the input, but
        with a depth size of num_classes
    """
    # st = time.time()
    # w = label.shape[0]
    # h = label.shape[1]
    # num_classes = len(class_dict)
    # x = np.zeros([w,h,num_classes])
    # unique_labels = sortedlist((class_dict.values()))
    # for i in range(0, w):
    #     for j in range(0, h):
    #         index = unique_labels.index(list(label[i][j][:]))
    #         x[i,j,index]=1
    # print("Time 1 = ", time.time() - st)

    # st = time.time()
    # https://stackoverflow.com/questions/46903885/map-rgb-semantic-maps-to-one-hot-encodings-and-vice-versa-in-tensorflow
    # https://stackoverflow.com/questions/14859458/how-to-check-if-all-values-in-the-columns-of-a-numpy-matrix-are-the-same
    semantic_map = []
    for colour in label_values:
        # colour_map = np.full((label.shape[0], label.shape[1], label.shape[2]), colour, dtype=int)
        equality = np.equal(label, colour)
        class_map = np.all(equality, axis = -1)
        semantic_map.append(class_map)
    semantic_map = np.stack(semantic_map, axis=-1)
    # print("Time 2 = ", time.time() - st)

    return semantic_map
    
def reverse_one_hot(image):
    """
    Transform a 2D array in one-hot format (depth is num_classes),
    to a 2D array with only 1 channel, where each pixel value is
    the classified class key.

    # Arguments
        image: The one-hot format image 
        
    # Returns
        A 2D array with the same width and hieght as the input, but
        with a depth size of 1, where each pixel value is the classified 
        class key.
    """
    # w = image.shape[0]
    # h = image.shape[1]
    # x = np.zeros([w,h,1])

    # for i in range(0, w):
    #     for j in range(0, h):
    #         index, value = max(enumerate(image[i, j, :]), key=operator.itemgetter(1))
    #         x[i, j] = index

    x = np.argmax(image, axis = -1)
    return x


def colour_code_segmentation(image, label_values):
    """
    Given a 1-channel array of class keys, colour code the segmentation results.

    # Arguments
        image: single channel array where each value represents the class key.
        label_values
        
    # Returns
        Colour coded image for segmentation visualization
    """

    # w = image.shape[0]
    # h = image.shape[1]
    # x = np.zeros([w,h,3])
    # colour_codes = label_values
    # for i in range(0, w):
    #     for j in range(0, h):
    #         x[i, j, :] = colour_codes[int(image[i, j])]
    
    colour_codes = np.array(label_values)
    x = colour_codes[image.astype(int)]

    return x

import csv
class_names_list, label_values =get_label_info("data/class_dict.csv")
class_names_string = ""
for class_name in class_names_list:
    if not class_name == class_names_list[-1]:
        class_names_string = class_names_string + class_name + ", "
    else:
        class_names_string = class_names_string + class_name

num_classes = len(label_values)

In [6]:
def pipeline_final(img):
    img = img[200:-500,:]

    img= cv2.resize(img, dsize=(608, 608))

    img = np.array([img])
    t = model.predict([img, img])
    output_image = reverse_one_hot(t)
    out_vis_image = colour_code_segmentation(output_image, label_values)
    a = cv2.cvtColor(np.uint8(out_vis_image[0]),1)
    b=cv2.cvtColor(np.uint8(img[0]),1)
    added_image = cv2.addWeighted(a,1,b,1,1)
    added_image= cv2.resize(added_image, dsize=(1080, 1220))

    return added_image


In [7]:
from moviepy.editor import VideoFileClip

# Reinitialize some global variables.
caches = [] 

output = 'output4.mp4'
clip1 = VideoFileClip("video2_Trim.mp4")
white_clip = clip1.fl_image(pipeline_final) 
%time white_clip.write_videofile(output, audio=False)

OSError: MoviePy error: the file video2_Trim.mp4 could not be found !
Please check that you entered the correct path.

In [13]:
img = load_image('Untitled.png')

In [14]:
img = np.array([cv2.resize(img, dsize=(608, 608), interpolation=cv2.INTER_CUBIC)])
print(img.shape)

(1, 608, 608, 3)


In [15]:
t  = model.predict([img, img])

In [16]:

output_image = reverse_one_hot(t)

In [17]:
out_vis_image = colour_code_segmentation(output_image, label_values)


In [18]:
cv2.imwrite("%s_pred.png"%("kiko"),cv2.cvtColor(np.uint8(out_vis_image[0]), cv2.COLOR_RGB2BGR))


True

In [19]:
cv2.imwrite("%s_pred.png"%("kiko2"),cv2.cvtColor(np.uint8(img[0]), cv2.COLOR_RGB2BGR))


True

In [20]:
import cv2

background = cv2.imread('kiko_pred.png') 
overlay = cv2.imread('kiko2_pred.png')

added_image = cv2.addWeighted(background,1,overlay,1,0)

added_image= cv2.resize(added_image, dsize=(608, 608), interpolation=cv2.INTER_CUBIC)
cv2.imwrite('combined.png', added_image)

True